In [46]:
# importation des packages
import numpy as np
import pandas as pd
import seaborn as sns

# importation des modules
import matplotlib.pyplot as plt
import scipy.stats as sps
import mysql.connector as sql

# magic
import warnings
warnings.filterwarnings('ignore')

db_connection = sql.connect(host='mysql.ozna.me', user='pnr1', password='groupesaepnr1', database='PNR', port='3306')

In [45]:
attributs = "espece, sexe, temperatureEau, dateObs, lieu_Lambert_X, lieu_Lambert_Y"
df = pd.read_sql(f"SELECT {attributs} FROM Obs_Hippocampe JOIN Observation ON obsH = idObs", con=db_connection)



,espece,sexe,temperatureEau,dateObs,lieu_Lambert_X,lieu_Lambert_Y
0,Syngnathus acus,femelle,10.0,2015-01-01,266355.27,6732979.36
1,Syngnathus acus,femelle,10.0,2015-01-01,266355.27,6732979.36
2,Syngnathus acus,femelle,10.0,2015-01-01,266355.27,6732979.36
3,Syngnathus acus,femelle,10.0,2015-01-01,266355.27,6732979.36
4,Syngnathus acus,femelle,10.0,2015-01-01,266355.27,6732979.36
...,...,...,...,...,...,...
1234,Hippocampus guttulatus,femelle,15.0,2017-05-19,267973.73,6733257.13
1235,Hippocampus guttulatus,male,15.0,2017-05-19,267973.73,6733257.13
1236,Syngnathus acus,femelle,15.0,2017-05-19,267941.04,6733383.28
1237,Syngnathus acus,male,15.0,2017-05-19,267941.04,6733383.28
